In [1]:
import os
import pandas as pd
import numpy as np
from tempfile import gettempdir

from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import Graph
from aequilibrae.paths import TrafficAssignment
from aequilibrae.paths.traffic_class import TrafficClass

In [5]:
net_file = "cube_links_gmns.csv"
net = pd.read_csv(net_file)
net

,from_node_id,to_node_id,link_id,length,directed,name,zone_id,capacity_scaling_factor,area_type,lanes,raw_capacity,speed,ftype,capacity,GEOMETRYSO,GEOMETRY_1,GEOMETRY_2,GEOMETRY_3,TL23_2032,TRN_2023
0,1,2070,1_2070,0.63722,0,NaN,1,1.0,Rural,7,10000.0,35.0,Centroid Connector,10000.0,1,1,1,1,0,0
1,2,2071,2_2071,0.72331,0,NaN,2,1.0,Rural,7,10000.0,35.0,Centroid Connector,10000.0,1,1,1,1,0,0
2,3,1171,3_1171,1.29217,0,NaN,22,1.0,Rural,7,10000.0,35.0,Centroid Connector,10000.0,1,1,1,1,0,0
3,3,2069,3_2069,0.94732,0,NaN,2,1.0,Rural,7,10000.0,35.0,Centroid Connector,10000.0,1,1,1,1,0,0
4,4,1994,4_1994,0.39404,0,NaN,4,1.0,Rural,7,10000.0,35.0,Centroid Connector,10000.0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,2806,373,2806_373,0.07162,0,NaN,364,1.0,Urban,7,10000.0,16.0,Centroid Connector,NaN,1,1,1,1,0,0
7789,2806,2200,2806_2200,0.12776,0,400 E,365,1.0,Urban,1,412.0,18.0,Local,NaN,1,1,1,1,0,0
7790,2806,2780,2806_2780,0.13510,0,400 E,365,1.0,Urban,1,412.0,18.0,Local,NaN,1,1,1,1,0,0
7791,2807,177,2807_177,0.20887,0,NaN,177,1.0,Urban,7,10000.0,16.0,Centroid Connector,NaN,1,1,1,1,0,0


In [21]:
zones = int(max(net.from_node_id.max(), net.to_node_id.max()))
zones
index = np.arange(zones) + 1
index

array([   1,    2,    3, ..., 2805, 2806, 2807])

In [16]:
dem = np.random.randint(low=0, high=100,size=(zones,zones))
dem

array([[99, 64, 39, ..., 92, 20, 59],
       [43, 91, 16, ..., 73, 64, 66],
       [56, 16, 66, ...,  3, 21, 58],
       ...,
       [30, 66, 34, ..., 37, 98, 31],
       [72, 19, 31, ..., 44, 20, 29],
       [44, 77, 70, ..., 51, 92, 26]])

In [22]:
aemfile = "aemfile.aem"
aem = AequilibraeMatrix()
kwargs = {'file_name': aemfile,
          'zones': zones,
          'matrix_names': ['matrix'],
          "memory_only": False}  # We'll save it to disk so we can use it later
aem.create_empty(**kwargs)
aem.matrix['matrix'][:,:] = dem[:,:]
aem.index[:] = index[:]

aem

In [57]:
net['travel_time'] = net['length']/net['speed']*60
net['a_node'] = net['from_node_id']
net['b_node'] = net['to_node_id']
net['direction'] = net['directed']
net['capacity'] = abs(net['capacity'].fillna(1)) + 1
net['travel_time'] = abs(net['travel_time'].fillna(1)) + 0.01

net

,from_node_id,to_node_id,link_id,length,directed,name,zone_id,capacity_scaling_factor,area_type,lanes,...,GEOMETRYSO,GEOMETRY_1,GEOMETRY_2,GEOMETRY_3,TL23_2032,TRN_2023,travel_time,a_node,b_node,direction
0,1,2070,1_2070,0.63722,0,NaN,1,1.0,Rural,7,...,1,1,1,1,0,0,1.102377,1,2070,0
1,2,2071,2_2071,0.72331,0,NaN,2,1.0,Rural,7,...,1,1,1,1,0,0,1.249960,2,2071,0
2,3,1171,3_1171,1.29217,0,NaN,22,1.0,Rural,7,...,1,1,1,1,0,0,2.225149,3,1171,0
3,3,2069,3_2069,0.94732,0,NaN,2,1.0,Rural,7,...,1,1,1,1,0,0,1.633977,3,2069,0
4,4,1994,4_1994,0.39404,0,NaN,4,1.0,Rural,7,...,1,1,1,1,0,0,0.685497,4,1994,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,2806,373,2806_373,0.07162,0,NaN,364,1.0,Urban,7,...,1,1,1,1,0,0,0.278575,2806,373,0
7789,2806,2200,2806_2200,0.12776,0,400 E,365,1.0,Urban,1,...,1,1,1,1,0,0,0.435867,2806,2200,0
7790,2806,2780,2806_2780,0.13510,0,400 E,365,1.0,Urban,1,...,1,1,1,1,0,0,0.460333,2806,2780,0
7791,2807,177,2807_177,0.20887,0,NaN,177,1.0,Urban,7,...,1,1,1,1,0,0,0.793262,2807,177,0


In [34]:
index

array([   1,    2,    3, ..., 2805, 2806, 2807])

In [58]:
g = Graph()
g.cost = network['travel_time'].values
g.capacity = network['capacity'].values
g.free_flow_time = network['travel_time'].values

g.network = network
g.prepare_graph(index)
g.set_graph("travel_time")
g.cost = np.array(g.cost, copy=True)
g.set_skimming(["travel_time"])
g.set_blocked_centroid_flows(False)
g.network["id"] = g.network.link_id

g

In [60]:
aem.computational_view(["matrix"])

assigclass = TrafficClass("car", g, aem)

assig = TrafficAssignment()

assig.set_classes([assigclass])
assig.set_vdf("BPR")
assig.set_vdf_parameters({"alpha": 0.5, "beta": 1.2})
assig.set_capacity_field("capacity")
assig.set_time_field("travel_time")
assig.set_algorithm("fw")
assig.max_iter = 100
assig.rgap_target = 1e-6

In [61]:
assig.execute()

In [62]:
print(assig.results())

              matrix_ab      matrix_ba    matrix_tot  Congested_Time_AB  \
link_id                                                                   
11707       1513.387732    1479.604217  2.992992e+03         300.330013   
11708       1485.469321    1507.315920  2.992785e+03         273.401214   
11713       1523.088291    1488.815431  3.011904e+03        1392.584698   
11719       1494.723365    1488.269003  2.982992e+03         240.312846   
12070     131015.492282  134927.811243  2.659433e+05          13.178599   
...                 ...            ...           ...                ...   
28042803  493817.018242  516963.860965  1.010781e+06      864483.963874   
28051206  279699.882091  308406.335449  5.881062e+05      113594.032244   
28052620  366693.384007  338253.343983  7.049467e+05      165471.425348   
28062200  477086.641670  462058.416306  9.391451e+05      269258.289679   
28062780  359537.734451  343298.298947  7.028360e+05      202518.545106   

          Congested_Time

In [63]:
print(assig.report())

    iteration      rgap     alpha warnings
0           1       inf  1.000000         
1           2  1.000000  0.499747         
2           3  0.990230  0.154212         
3           4  0.989951  0.109455         
4           5  0.975029  0.159613         
..        ...       ...       ...      ...
95         96  0.050427  0.006994         
96         97  0.049232  0.007604         
97         98  0.048101  0.007908         
98         99  0.048038  0.007391         
99        100  0.047151  0.007622         

[100 rows x 4 columns]
